## Prep single series

In [2]:
from gluonts.dataset.repository import get_dataset, dataset_names
from gluonts.dataset.util import to_pandas

print(f"Available datasets: {dataset_names}")

Available datasets: ['constant', 'exchange_rate', 'solar-energy', 'electricity', 'traffic', 'exchange_rate_nips', 'electricity_nips', 'traffic_nips', 'solar_nips', 'wiki2000_nips', 'wiki-rolling_nips', 'taxi_30min', 'kaggle_web_traffic_with_missing', 'kaggle_web_traffic_without_missing', 'kaggle_web_traffic_weekly', 'm1_yearly', 'm1_quarterly', 'm1_monthly', 'nn5_daily_with_missing', 'nn5_daily_without_missing', 'nn5_weekly', 'tourism_monthly', 'tourism_quarterly', 'tourism_yearly', 'cif_2016', 'london_smart_meters_without_missing', 'wind_farms_without_missing', 'car_parts_without_missing', 'dominick', 'fred_md', 'pedestrian_counts', 'hospital', 'covid_deaths', 'kdd_cup_2018_without_missing', 'weather', 'm3_monthly', 'm3_quarterly', 'm3_yearly', 'm3_other', 'm4_hourly', 'm4_daily', 'm4_weekly', 'm4_monthly', 'm4_quarterly', 'm4_yearly', 'm5', 'uber_tlc_daily', 'uber_tlc_hourly', 'airpassengers', 'australian_electricity_demand', 'electricity_hourly', 'electricity_weekly', 'rideshare_wit

In [1]:
from gluonts.dataset.repository import get_dataset, dataset_names
from gluonts.dataset.util import to_pandas

# print(f"Available datasets: {dataset_names}")
dataset = get_dataset("kaggle_web_traffic_without_missing")

/home/reshawn/miniconda3/envs/clfr/lib/python3.11/site-packages/gluonts/json.py:102: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  warnings.warn(
Download kaggle_web_traffic_dataset_without_missing_values.zip:: 138MB [00:26, 5.51MB/s]                            
creating json files: 100%|██████████| 145063/145063 [00:01<00:00, 117175.38it/s]


In [34]:
dataset.test

Map(fn=<gluonts.dataset.common.ProcessDataEntry object at 0x7d8bcc77c7d0>, iterable=JsonLinesFile(path=PosixPath('/home/reshawn/.gluonts/datasets/solar-energy/test/data.json.gz'), start=0, n=None, line_starts=[0, -16642, 30420, 11963, -6552, -22993, 26324, 7988, -10536, -29270, 17701, -766, -20094, 26799, 8473, -10360, -28824, 17955, -516, -19346, 27843, 9382, -9343, -28038, 20247, 1660, -16561, 30303, 12099, -5101, -23418, 24561, 7236, -11617, -30044, 16763, -1725, -20486, 26552, 8057, -10572, -27626, 19502, 933, -17442, 30466, 12582, -5430, -24182, 22689, 3872, -14554, 32515, 13885, -4413, -22598, 23928, 5529, -12842, -30921, 16599, -1937, -20652, 26006, 7653, -10893, -29776, 17279, -1126, -18013, 30335, 12711, -5775, -23821, 25201, 6647, -11882, -30049, 17311, -1059, -19917, 27918, 9357, -8877, -27651, 19630, 1086, -17881, 29007, 10638, -8078, -25922, 21093, 2067, -16147, 30330, 11777, -6263, -24823, 21915, 3489, -15533, 31546, 12802, -5864, -22871, 25390, 6467, -11847, -29850, 1689

In [2]:
# to reload local packages when debugging
%load_ext autoreload
%autoreload 2
%cd /mnt/c/Users/resha/Documents/Github/balancing_framework/
from monash_data_utils import convert_tsf_to_dataframe, monash_df_to_gluonts_dataset
import os
import pandas as pd

dataset_name = "kaggle_web_traffic_dataset_without_missing_values" # m4_hourly_dataset
dataset = None
monash_dir = "monash_data"

if os.path.exists(f"{monash_dir}/{dataset_name}.tsf"):
    loaded_data, frequency, forecast_horizon, contain_missing_values, contain_equal_length = convert_tsf_to_dataframe(f"{monash_dir}/{dataset_name}.tsf")
    dataset = monash_df_to_gluonts_dataset(loaded_data, frequency)

if dataset is None:
    if dataset_name in dataset_names:
        dataset = get_dataset(dataset_name)
    else:
        raise ValueError(f"Dataset {dataset_name} not found in gluonts availables or local monash files.")
    

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/mnt/c/Users/resha/Documents/Github/balancing_framework


: 

In [35]:
forecast_horizon

In [10]:
entry['target']

array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)

In [3]:
from gluonts.dataset.repository import get_dataset, dataset_names
from gluonts.dataset.util import to_pandas

# print(f"Available datasets: {dataset_names}")
dataset = get_dataset("solar-energy")
series_num = -1

for entry in dataset.test:
    # print(entry['item_id'])
    if entry['item_id'] == 3:
        break
series_num = entry['item_id']

import pandas as pd
import numpy as np

def adjust_split_point(series, threshold=0.05, max_iterations=1000):
    split_point = np.median(series)
    increment = 0.01  # Initial increment
    iterations = 0

    while iterations < max_iterations:
        binary_series = (series > split_point).astype(int)
        balance = binary_series.mean()[0]

        # Check if balance is within the threshold
        if abs(balance - 0.5) <= threshold:
            break

        # Adjust increment based on how far off the balance is
        distance_from_balance = abs(balance - 0.5)
        increment = max(0.001, increment * (1 + distance_from_balance))  # Scale increment

        # Adjust split point based on balance
        if balance < 0.5:
            split_point -= increment
        else:
            split_point += increment

        iterations += 1

    return split_point

def derive_feats(df):
    # Define window sizes for rolling calculations
    window_sizes = [2, 3, 4, 5]

    # Calculate rolling window features for different window sizes
    for window_size in window_sizes:
        df[f'rolling_mean_{window_size}'] = df['values'].rolling(window=window_size).mean()
        df[f'rolling_sum_{window_size}'] = df['values'].rolling(window=window_size).sum()
        df[f'rolling_min_{window_size}'] = df['values'].rolling(window=window_size).min()
        df[f'rolling_max_{window_size}'] = df['values'].rolling(window=window_size).max()
        df[f'rolling_std_{window_size}'] = df['values'].rolling(window=window_size).std()

    # Calculate additional features
    df['lag_1'] = df['values'].shift(1)  # Previous value
    df['lag_2'] = df['values'].shift(2)  # Value two steps back
    df['lag_3'] = df['values'].shift(3)  # Value three steps back
    df['lag_4'] = df['values'].shift(4)  # Value four steps back
    df['difference'] = df['values'].diff()  # Difference from the previous value
    df['percentage_change'] = df['values'].pct_change()  # Percentage change from the previous value
    df['cumulative_sum'] = df['values'].cumsum()  # Cumulative sum
    df['cumulative_mean'] = df['values'].expanding().mean()  # Cumulative mean
    df['cumulative_max'] = df['values'].cummax()  # Cumulative maximum
    df['cumulative_min'] = df['values'].cummin()  # Cumulative minimum
    df.dropna(inplace=True)

    return df.reset_index(drop=True)




df = pd.DataFrame(entry['target'])
series = df[[0]].copy()
changes = series.diff().dropna()

split_point = adjust_split_point(changes)
binary_series = (changes > split_point).astype(int)

df = df.iloc[:-1].copy() # reflect diff drop of row 1 on original df
df['label'] = binary_series.values
df.rename(columns={0: 'values'}, inplace=True)

df = derive_feats(df)

/home/reshawn/miniconda3/envs/clfr/lib/python3.11/site-packages/gluonts/json.py:102: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  warnings.warn(


In [13]:
df = df.replace([np.inf, -np.inf], np.nan).dropna()

In [12]:
X.dropna()['cumulative_max'].iloc[0]

NameError: name 'X' is not defined

## Run frac diff

In [5]:
# to reload local packages when debugging
%load_ext autoreload
%autoreload 2

%cd /mnt/c/Users/resha/Documents/Github/balancing_framework/

import pickle
import numpy as np



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/mnt/c/Users/resha/Documents/Github/balancing_framework


In [ ]:
from fracdiff import frac_diff_bestd, frac_diff_ffd

X = df.drop(columns=['label']) 
y = df['label']

X, fd_change_pct = frac_diff_bestd(X)

X.dropna(inplace=True)
y = y.iloc[X.iloc[0].name:]
X.reset_index(drop=True, inplace=True)
y.reset_index(drop=True, inplace=True)

## Run framework eval

In [8]:
X_original = df.drop(columns=['label']) 
y_original = df['label']


from framework import run_measurements, viz

dataset_name = 'md4_1'
model_name = 'random_forest'
# lower bounds to chunk size
# min to run, chunk_size > X.shape(1) * 10
# min for 10 splits, chunk_size > X.shape(0) /10 ;;; can adjust this min, unsure atm how many series fails condition
# upper bound to chunk size
# max num of splits, 100, 100 > (X.shape(0) / chunk_size) therefore default to 1% chunk size, using the lower bound
chunk_size = int(X.shape[0] * 0.01)
min_splits = 3
if chunk_size < (X.shape[1] * 10):
    chunk_size = X.shape[1] * 10
if chunk_size < (X.shape[0] / min_splits):
    print(f'series length {X.shape[0]} with {X.shape[1]} feats does not meet criteria of >=10*n_features and a resulting count of splits > min_splits={min_splits}.')
    pass
# cold_start_size = 10_000
num_runs = 10
data_form = 'original'


a,c,p = run_measurements(X, y, chunk_size, dataset_name, model_name, num_runs=num_runs, frac_diff=False)


####################################### Save Results and Visualizations ###############################################################################
# import os
# base_dir = '/mnt/c/Users/resha/Documents/Github/balancing_framework/results/notebooks_alt_data'
# dir_ext = f'{dataset_name}_{series_num} {data_form} chunk_size={chunk_size} num_runs={num_runs} {model_name}'
# save_dir = os.path.join(base_dir, dir_ext)
# if not os.path.exists(save_dir):
#     os.mkdir(save_dir)
# with open(f'{save_dir}/adaptation_results_{data_forms}.pkl', 'wb') as f:
#     pickle.dump(a, f)
# with open(f'{save_dir}/consolidation_results_{data_forms}.pkl', 'wb') as f:
#     pickle.dump(c, f)

# end = time.time()
# print(f"Runtime: {(end - start) / 60} minutes")

# viz(a, c, metric='accuracy', title=data_form, dir=save_dir) 
# viz(a, c, metric='f1', title=data_form, dir=save_dir) 

# base_dir = '/mnt/c/Users/resha/Documents/Github/balancing_framework/notebooks_alt_data'
# dir_ext = f'{dataset_name}_{series_num} {data_form} chunk_size={chunk_size} num_runs={num_runs} {model_name}'
# save_dir = f'{base_dir}/{dir_ext}'
# viz(a, c, metric='accuracy', title=data_form, dir=save_dir) 
# viz(a, c, metric='f1', title=data_form, dir=save_dir) 

RUNNING ADAPTATION MEASURE


  0%|          | 0/3 [00:00<?, ?it/s][I 2025-08-05 11:08:07,137] A new study created in memory with name: no-name-b44357f0-0fb7-4b78-90d4-f2744c3fb58d
[I 2025-08-05 11:08:07,252] Trial 0 finished with value: 0.9107142857142857 and parameters: {'n_estimators': 29, 'max_depth': 14}. Best is trial 0 with value: 0.9107142857142857.
[I 2025-08-05 11:08:07,292] Trial 1 finished with value: 0.9464285714285714 and parameters: {'n_estimators': 7, 'max_depth': 14}. Best is trial 1 with value: 0.9464285714285714.


Tuning run 1 of 3


[I 2025-08-05 11:08:07,352] Trial 2 finished with value: 0.9107142857142857 and parameters: {'n_estimators': 12, 'max_depth': 14}. Best is trial 1 with value: 0.9464285714285714.
[I 2025-08-05 11:08:07,515] Trial 3 finished with value: 0.9285714285714286 and parameters: {'n_estimators': 48, 'max_depth': 13}. Best is trial 1 with value: 0.9464285714285714.
[I 2025-08-05 11:08:07,616] Trial 4 finished with value: 0.9107142857142857 and parameters: {'n_estimators': 23, 'max_depth': 28}. Best is trial 1 with value: 0.9464285714285714.
[I 2025-08-05 11:08:07,731] Trial 5 finished with value: 0.9107142857142857 and parameters: {'n_estimators': 27, 'max_depth': 32}. Best is trial 1 with value: 0.9464285714285714.
[I 2025-08-05 11:08:07,822] Trial 6 finished with value: 0.9107142857142857 and parameters: {'n_estimators': 22, 'max_depth': 2}. Best is trial 1 with value: 0.9464285714285714.
[I 2025-08-05 11:08:07,873] Trial 7 finished with value: 0.9107142857142857 and parameters: {'n_estimators

Optimization Time: 0.01693574791715946 minutes
Training run 1 of 3
Run 0 Accuracy: 0.9355
Run 1 Accuracy: 0.9677
Run 2 Accuracy: 0.9355
Run 3 Accuracy: 0.9677
Run 4 Accuracy: 0.9032
Run 5 Accuracy: 0.9355
Run 6 Accuracy: 0.9032
Run 7 Accuracy: 0.9355


...

Run 9 Accuracy: 0.9516
Tuning run 3 of 3


[I 2025-08-05 11:08:17,268] Trial 3 finished with value: 0.9696969696969697 and parameters: {'n_estimators': 13, 'max_depth': 30}. Best is trial 3 with value: 0.9696969696969697.
[I 2025-08-05 11:08:17,336] Trial 4 finished with value: 0.9545454545454546 and parameters: {'n_estimators': 10, 'max_depth': 7}. Best is trial 3 with value: 0.9696969696969697.
[I 2025-08-05 11:08:17,564] Trial 5 finished with value: 0.9545454545454546 and parameters: {'n_estimators': 50, 'max_depth': 32}. Best is trial 3 with value: 0.9696969696969697.
[I 2025-08-05 11:08:17,598] Trial 6 finished with value: 0.946969696969697 and parameters: {'n_estimators': 4, 'max_depth': 15}. Best is trial 3 with value: 0.9696969696969697.
[I 2025-08-05 11:08:17,700] Trial 7 finished with value: 0.9545454545454546 and parameters: {'n_estimators': 21, 'max_depth': 19}. Best is trial 3 with value: 0.9696969696969697.
[I 2025-08-05 11:08:17,868] Trial 8 finished with value: 0.946969696969697 and parameters: {'n_estimators': 

Optimization Time: 0.016734670767133746 minutes
Training run 3 of 3
Run 0 Accuracy: 0.9595
Run 1 Accuracy: 0.9595
Run 2 Accuracy: 0.9730
Run 3 Accuracy: 0.9595
Run 4 Accuracy: 0.9595
Run 5 Accuracy: 0.9595
Run 6 Accuracy: 0.9595
Run 7 Accuracy: 0.9730


100%|██████████| 3/3 [00:05<00:00,  1.85s/it]

Run 8 Accuracy: 0.9459
Run 9 Accuracy: 0.9459
